# Concordia <> Mastodon Example

## Introduction

**Purpose of the Simulation**

This notebook demonstrates a generative agent simulation of a Mastodon instance using the Concordia framework. 
The simulation models user behavior on a local Mastodon server, showcasing how AI agents interact in a 
decentralized social media environment.

## Setup and imports


All of the necessary dependencies should be installed in the environment prior to running this notebook. If not, install them using poetry:

```bash
poetry install
```


Alternatively, you may install the library in editable mode so that we can make changes to the code if necessary:

```bash
git clone https://github.com/gdm-concordia/concordia.git
cd concordia
pip install -e . --config-settings editable_mode=compat
```

In [1]:
%load_ext autoreload
%autoreload 2

import concurrent.futures
import datetime
import os
import random
import warnings
import json

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    import sentence_transformers
import sys

sys.path.insert(0, "../concordia")
# import concordia_dev
# print(concordia_dev.__file__)
from concordia import components as generic_components
from concordia.agents import deprecated_agent as basic_agent
from concordia.associative_memory import (
    associative_memory,
    blank_memories,
    formative_memories,
    importance_function,
)
from concordia.clocks import game_clock
from concordia.components.agent import to_be_deprecated as components
from concordia.components import game_master as gm_components

# from concordia_dev.environment import game_master
# from concordia_dev.language_model import amazon_bedrock_model, gpt_model
from concordia.environment import game_master
from concordia.language_model import amazon_bedrock_model, gpt_model, ollama_model
from concordia.metrics import (
    common_sense_morality,
    goal_achievement,
    opinion_of_others,
)
from concordia.utils import html as html_lib
from concordia.utils import measurements as measurements_lib
from IPython import display

from mastodon_sim.concordia import apps, triggering

from mastodon_sim.mastodon_ops import check_env, get_public_timeline, print_timeline, reset_users
from mastodon_sim.mastodon_utils import get_users_from_env
import re

## Mastodon Server Interaction Setting

Decide whether this example should perform real operations on the Mastodon server. Note that this requires being able to successfully run the `Mastodon.ipynb` example as a prerequisite.

In [2]:
USE_MASTODON_SERVER = True

## Check environment

Check for a `.env` file with the nessesary environment variables. You should see an output similar to this if successful:

```log
2024-07-17T11:38:17.213066-0400 INFO Successfully loaded .env file.
2024-07-17T11:38:17.214151-0400 INFO API_BASE_URL: https://social-sandbox.com
2024-07-17T11:38:17.214581-0400 INFO EMAIL_PREFIX: austinmw89
2024-07-17T11:38:17.215056-0400 INFO MASTODON_CLIENT_ID: N*****************************************E
2024-07-17T11:38:17.215477-0400 INFO MASTODON_CLIENT_SECRET: b*****************************************s
2024-07-17T11:38:17.215877-0400 INFO USER0005_PASSWORD: a******************************a
2024-07-17T11:38:17.216202-0400 INFO USER0002_PASSWORD: 8******************************b
2024-07-17T11:38:17.216655-0400 INFO USER0001_PASSWORD: 9******************************5
2024-07-17T11:38:17.217020-0400 INFO USER0004_PASSWORD: 7******************************e
2024-07-17T11:38:17.217323-0400 INFO USER0003_PASSWORD: f******************************6
```

In [3]:
if USE_MASTODON_SERVER:
    check_env()

2024-12-14T22:53:43.764604+0000 INFO Successfully loaded .env file.
2024-12-14T22:53:43.765511+0000 INFO API_BASE_URL: https://mastodon.genexergy.org
2024-12-14T22:53:43.766343+0000 INFO EMAIL_PREFIX: gefi
2024-12-14T22:53:43.766820+0000 INFO MASTODON_CLIENT_ID: D*****************************************E
2024-12-14T22:53:43.767274+0000 INFO MASTODON_CLIENT_SECRET: A*****************************************s
2024-12-14T22:53:43.768913+0000 INFO USER0159_PASSWORD: 7******************************4
2024-12-14T22:53:43.770855+0000 INFO USER0121_PASSWORD: 0******************************9
2024-12-14T22:53:43.771421+0000 INFO USER0027_PASSWORD: 5******************************e
2024-12-14T22:53:43.772003+0000 INFO USER0013_PASSWORD: 8******************************8
2024-12-14T22:53:43.772898+0000 INFO USER0046_PASSWORD: e******************************d
2024-12-14T22:53:43.774111+0000 INFO USER0152_PASSWORD: 6******************************9
2024-12-14T22:53:43.774612+0000 INFO USER0151_PASSWORD

## Clear Mastodon server
Deletes from https://social-sandbox.com all posts, follows, likes, etc. from all users in the .env file.

TODO: The Mastodon server's [default API rate limits](https://docs.joinmastodon.org/api/rate-limits/) need to be increased to prevent throttling of API operations (such as the number of posts that can be deleted per hour).

In [4]:
if USE_MASTODON_SERVER:
    # Get all users from .env file
    users = get_users_from_env()

    reset_users(users, skip_confirm=True, parallel=True)

    assert not len(get_public_timeline(limit=None)), "All posts not cleared"

2024-12-14T22:53:44.770496+0000 INFO Starting comprehensive reset process for user: user0001
2024-12-14T22:53:44.785134+0000 INFO Starting comprehensive reset process for user: user0004
2024-12-14T22:53:44.817232+0000 INFO Starting comprehensive reset process for user: user0005
2024-12-14T22:53:44.986086+0000 INFO Starting comprehensive reset process for user: user0003
2024-12-14T22:53:44.996180+0000 INFO Starting comprehensive reset process for user: user0002
2024-12-14T22:53:45.012469+0000 INFO Starting comprehensive reset process for user: user0007
2024-12-14T22:53:45.051475+0000 INFO Starting comprehensive reset process for user: user0008
2024-12-14T22:53:45.150464+0000 INFO Starting comprehensive reset process for user: user0006
2024-12-14T22:53:45.416150+0000 ERROR Error: invalid literal for int() with base 10: '66, 299'
2024-12-14T22:53:45.446282+0000 ERROR Error: invalid literal for int() with base 10: '66, 299'
2024-12-14T22:53:45.470643+0000 ERROR Error: invalid literal for i

## Set LLM and embedding model

In [5]:
pwd

'/workspaces/mastodon-sim-2/notebooks'

In [6]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
os.chdir("../../mastodon-sim/")

In [8]:
os.chdir("notebooks/")

In [9]:
MODEL_NAME = "ollama"
# MODEL_NAME = "gpt-4o-mini"
# MODEL_NAME = "sonnet"

if "ollama" in MODEL_NAME:
    model = ollama_model.OllamaLanguageModel(model_name="gemma2:27b")
elif "gpt" in MODEL_NAME:
    GPT_API_KEY = os.getenv("OPENAI_API_KEY")
    if not GPT_API_KEY:
        raise ValueError("GPT_API_KEY is required.")
    model = gpt_model.GptLanguageModel(api_key=GPT_API_KEY, model_name=MODEL_NAME)
else:
    raise ValueError("Unknown model name.")

In [10]:
def write_agent_config_for_independent_sim(agent_config_filename):
    agent_names = [
        "Bill Fredrickson",
        "Maria Gutierrez",
        "Glenn Patterson",
        "Denise Schmidt",
        "Roger Davis",
        "Erica Fitzgerald",
        "Liam Schwartz",
        "Olivia Thompson",
        "Robert Johnson",
        "Janet Thompson",
        "William Davis",
        "Jessica Nguyen",
        "Mark Rodriguez",
        "Emily Jacobs",
        "Ethan Lee",
        "Sophia Patel",
        "Ryan O'Connor",
        "Maggie Chen",
        "Lucas Kim",
        "Nina Patel",
    ]
    genders = [
        "male",
        "female",
        "male",
        "female",
        "male",
        "female",
        "male",
        "female",
        "male",
        "female",
        "male",
        "female",
        "male",
        "female",
        "male",
        "female",
        "male",
        "female",
        "male",
        "female",
    ]
    big5 = [
        [3, 8, 6, 5, 4],
        [9, 7, 8, 8, 5],
        [1, 5, 7, 2, 9],
        [1, 6, 8, 2, 7],
        [2, 3, 7, 2, 8],
        [8, 6, 9, 3, 7],
        [8, 5, 8, 3, 7],
        [5, 6, 9, 2, 8],
        [5, 7, 4, 6, 5],
        [6, 7, 5, 7, 4],
        [4, 6, 5, 6, 4],
        [7, 6, 5, 7, 5],
        [6, 7, 5, 7, 4],
        [7, 6, 5, 8, 6],
        [6, 8, 6, 6, 4],
        [7, 8, 6, 7, 4],
        [5, 7, 5, 6, 5],
        [6, 7, 6, 8, 4],
        [8, 6, 5, 6, 5],
        [7, 8, 7, 8, 4],
    ]
    big5_keys = ["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]
    candidate_goal = "Win the election and become the mayor of Storhampton."
    active_voter_goal = "Have a good day and vote in the election."
    active_voter_context = "doesn't care about the environment, only about having a stable job"
    Marias_policy_proposals = [
        "increasing environmental regulation of local industries to improve the health of the local environment"
    ]
    Bills_policy_proposals = [
        "lowering taxes on local industries to save jobs",
        "providing subsidies to attract green industries and create jobs to help grow the economy",
    ]
    agent_config = {}
    agent_config["agents"] = []
    for ait, agent_name in enumerate(agent_names):
        agent_dict = {}
        agent_dict["name"] = agent_name
        agent_dict["gender"] = genders[ait]
        agent_dict["traits"] = dict(zip(big5_keys, big5[ait]))
        if ait < 2:
            agent_dict["role"] = "candidate"
            agent_dict["goal"] = candidate_goal
            if agent_dict["name"] == "Maria Gutierrez":
                agent_dict["policy_proposals"] = Marias_policy_proposals
            else:
                agent_dict["policy_proposals"] = Bills_policy_proposals
            agent_dict["context"] = ""
        else:
            agent_dict["role"] = "active_voter"
            agent_dict["context"] = active_voter_context
            agent_dict["goal"] = active_voter_goal
        agent_dict["party"] = ""
        agent_dict["seed_toot"] = ""
        agent_config["agents"].append(agent_dict)
    agent_config["shared_memories_template"] = [
        "You are a user on Storhampton.social, a Mastodon instance created for the residents of Storhampton.",
        "Storhampton is a small town with a population of approximately 2,500 people. Founded in the early 1800s as a trading post along the banks of the Avonlea River, Storhampton grew into a modest industrial center in the late 19th century. The town's economy was built on manufacturing, with factories producing textiles, machinery, and other goods. Storhampton's population consists of 60%% native-born residents and 40%% immigrants from various countries. Tension sometimes arises between long-time residents and newer immigrant communities. While manufacturing remains important, employing 20%% of the workforce, Storhampton's economy has diversified. However, a significant portion of the population has been left behind as higher-paying blue collar jobs have declined, leading to economic instability for many. The poverty rate stands at 15%.",
        "Mayoral Elections: The upcoming mayoral election in Storhampton has become a heated affair.",
        "Social media has emerged as a key battleground in the race, with both candidates actively promoting themselves and engaging with voters. Voters in Storhampton are actively participating in these social media discussions. Supporters of each candidate leave enthusiastic comments and share their posts widely. Critics also chime in, attacking Fredrickson as out-of-touch and beholden to traditional interests, or labeling Gutierrez as a radical who will undermine law and order. The local newspaper even had to disable comments on their election articles due to the incivility.",
    ]
    agent_config["mastodon_usage_instructions"] = [
        "To share content on Mastodon, you write a 'toot' (equivalent to a tweet or post).",
        "Toots can be up to 500 characters long, allowing for more detailed expressions than some other platforms.",
        "Your home timeline shows toots from people you follow and boosted (reblogged) content.",
        "You can reply to toots, creating threaded conversations.",
        "Favorite (like) toots to show appreciation or save them for later.",
        "Boost (reblog) toots to share them with your followers.",
        "You can mention other users in your toots using their @username.",
        "Follow other users to see their public and unlisted toots in your home timeline.",
        "You can unfollow users if you no longer wish to see their content.",
        "Your profile can be customized with a display name and bio.",
        "You can block users to prevent them from seeing your content or interacting with you.",
        "Unblocking a user reverses the effects of blocking.",
    ]
    agent_config["num_agents"] = len(agent_names)

    # Convert and write JSON object to file
    with open(agent_config_filename, "w") as outfile:
        json.dump(agent_config, outfile, indent=4)


agent_config_filename = "independent_configs_test1_Bill_bias.json"
write_agent_config_for_independent_sim(agent_config_filename)

In [11]:
import json

# Setup sentence encoder
st_model = sentence_transformers.SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
embedder = lambda x: st_model.encode(x, show_progress_bar=False)  # noqa: E731
# with open('agents_data.json', 'r') as file:
with open(agent_config_filename, "r") as file:
    data = json.load(file)
data["candidate_info"] = []
for agent in data["agents"]:
    if agent["role"] == "candidate":
        candidate_info = f"{agent['name']} campaigns on {' and '.join(agent['policy_proposals'])}."
        data["candidate_info"].append(candidate_info)
        agent["context"] = agent["context"] + candidate_info
    else:
        agent["context"] = f"{agent['name']} is a person who {agent['context']}"

/opt/mastodon-sim-env/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Configuring the generic knowledge of the players and the game master (GM)

In [12]:
shared_memories = (
    data["shared_memories_template"] + data["candidate_info"] + data["mastodon_usage_instructions"]
)

In [13]:
# The generic context will be used for the NPC context. It reflects general
# knowledge and is possessed by all characters.
shared_context = model.sample_text(
    "Summarize the following passage in a concise and insightful fashion. "
    + "Make sure to include information about Mastodon:\n"
    + "\n".join(shared_memories)
    + "\nSummary:",
    max_tokens=2048,
)

print(shared_context)
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()

Storhampton, a town of 2,500 residents founded in the early 1800s, is grappling with economic and social challenges as it transitions from an industrial center to a more diversified economy. The upcoming mayoral election highlights these tensions, with candidates Fredrickson and Gutierrez offering contrasting visions for the town's future.  

Fredrickson focuses on preserving traditional industries through tax breaks and subsidies for green jobs. Gutierrez emphasizes stricter environmental regulations to improve public health. This debate has spilled onto social media platforms like Mastodon, where residents passionately engage in discussions about the candidates and their platforms. 

Mastodon, a decentralized social network, allows users to share longer "toots" (up to 500 characters) and participate in threaded conversations. Users can follow others, boost content, and customize their profiles. The platform's structure fosters community engagement and in-depth dialogue. 



In [14]:
# Make the clock
time_step = datetime.timedelta(minutes=30)

SETUP_TIME = datetime.datetime(year=2024, month=10, day=1, hour=8)  # noqa: DTZ001

START_TIME = datetime.datetime(year=2024, month=10, day=1, hour=8)  # noqa: DTZ001

clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME, step_sizes=[time_step, datetime.timedelta(seconds=10)]
)

## Functions to build the players

In [15]:
blank_memory_factory = blank_memories.MemoryFactory(
    model=model,
    embedder=embedder,
    importance=importance_model.importance,
    clock_now=clock.now,
)

formative_memory_factory = formative_memories.FormativeMemoryFactory(
    model=model,
    shared_memories=shared_memories,
    blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)

# All players get the same `measurements` object.
measurements = measurements_lib.Measurements()

In [16]:
# Build agent function


def build_agent(
    agent_config, measurements: measurements_lib.Measurements, player_names: list[str]
) -> tuple:
    """Build an agent based on the given configuration."""
    mem = formative_memory_factory.make_memories(agent_config)

    instructions = generic_components.constant.ConstantComponent(
        state=(
            f"The instructions for how to play the role of {agent_config.name} are as "
            "follows. This is a social science experiment studying how well you "
            f"play the role of a character named {agent_config.name}. The experiment "
            "is structured as a tabletop roleplaying game (like dungeons and "
            "dragons). However, this is a serious social science experiment and "
            "simulation, not just a game. The goal is to be realistic. It is "
            f"important to play the role of a person like {agent_config.name} as "
            "accurately as possible, i.e., by responding in ways that you think "
            f"it is likely a person like {agent_config.name} would respond, and taking "
            f"into account all information about {agent_config.name} that you have. "
            "You will control just this one character. The game master will describe "
            "the current situation, and based on that information, you will suggest "
            "actions for your character. The game master will track the state of the "
            "world and keep it consistent as time passes in the simulation and as "
            "participants take actions and change things in their world. This experiment "
            "need not be fun for the participants. Always use third-person "
            "limited perspective, even when speaking directly to the game master."
        ),
        name="role playing instructions\n",
    )

    election_information = generic_components.constant.ConstantComponent(
        state=("\n".join(data["candidate_info"])), name="critical election information\n"
    )

    time = generic_components.report_function.ReportFunction(
        name="Current time",
        function=clock.current_time_interval_str,
    )

    somatic_state = components.somatic_state.SomaticState(model, mem, agent_config.name, clock.now)
    identity = components.identity.SimIdentity(
        model=model,
        memory=mem,
        agent_name=agent_config.name,
        clock_now=clock.now,
    )

    current_obs = components.observation.Observation(
        agent_name=agent_config.name,
        clock_now=clock.now,
        memory=mem,
        timeframe=time_step * 1,
        component_name="current observations",
    )
    summary_obs = components.observation.ObservationSummary(
        agent_name=agent_config.name,
        model=model,
        clock_now=clock.now,
        memory=mem,
        timeframe_delta_from=datetime.timedelta(hours=4),
        timeframe_delta_until=time_step * 1,
        components=[identity],
        component_name="summary of observations",
    )

    self_perception = components.self_perception.SelfPerception(
        name=f"answer to what kind of person is {agent_config.name}",
        model=model,
        memory=mem,
        agent_name=agent_config.name,
        clock_now=clock.now,
    )
    relevant_memories = components.all_similar_memories.AllSimilarMemories(
        name="relevant memories",
        model=model,
        memory=mem,
        agent_name=agent_config.name,
        components=[summary_obs, self_perception],
        clock_now=clock.now,
        num_memories_to_retrieve=10,
    )

    situation_perception = components.situation_perception.SituationPerception(
        name=(f"answer to what kind of situation is {agent_config.name} in " + "right now"),
        model=model,
        memory=mem,
        agent_name=agent_config.name,
        components=[current_obs, somatic_state, summary_obs],
        clock_now=clock.now,
    )
    person_by_situation = components.person_by_situation.PersonBySituation(
        name=(
            f"answer to what would a person like {agent_config.name} do in a "
            + "situation like this"
        ),
        model=model,
        memory=mem,
        agent_name=agent_config.name,
        clock_now=clock.now,
        components=[self_perception, situation_perception],
        verbose=True,
    )
    persona = components.sequential.Sequential(
        name="persona",
        components=[
            self_perception,
            situation_perception,
            person_by_situation,
        ],
    )

    reflection = components.dialectical_reflection.DialecticalReflection(
        name="reflection",
        model=model,
        memory=mem,
        agent_name=agent_config.name,
        intuition_components=[self_perception],
        thinking_components=[persona],
        clock_now=clock.now,
        num_memories_to_retrieve=3,
        verbose=True,
    )

    initial_goal_component = generic_components.constant.ConstantComponent(
        state=agent_config.goal, name="overarching goal"
    )

    plan = components.plan.SimPlan(
        model=model,
        memory=mem,
        agent_name=agent_config.name,
        clock_now=clock.now,
        components=[instructions, identity, initial_goal_component, somatic_state, persona],
        goal=initial_goal_component,
        verbose=False,
    )

    goal_metric = goal_achievement.GoalAchievementMetric(
        model=model,
        player_name=agent_config.name,
        player_goal=agent_config.goal,
        clock=clock,
        name="Goal Achievement",
        measurements=measurements,
        channel="goal_achievement",
        verbose=True,
    )
    morality_metric = common_sense_morality.CommonSenseMoralityMetric(
        model=model,
        player_name=agent_config.name,
        clock=clock,
        name="Morality",
        verbose=True,
        measurements=measurements,
        channel="common_sense_morality",
    )

    agent = basic_agent.BasicAgent(
        model=model,
        agent_name=agent_config.name,
        clock=clock,
        verbose=True,
        components=[
            instructions,
            election_information,
            identity,
            persona,
            plan,
            somatic_state,
            summary_obs,
            current_obs,
            relevant_memories,
            reflection,
            time,
            goal_metric,
            morality_metric,
        ],
        update_interval=time_step,
    )

    # reputation_metric = opinion_of_others.OpinionOfOthersMetric(
    #     model=model,
    #     player_name=agent_config.name,
    #     player_names=player_names,
    #     context_fn=agent.state,
    #     clock=clock,
    #     name="Opinion",
    #     verbose=True,
    #     measurements=measurements,
    #     channel="opinion_of_others",
    #     question="What is {opining_player}'s opinion of {of_player}?",
    # )
    # agent.add_component(reputation_metric)

    return agent, mem

In [17]:
from concordia.typing import entity
from concordia.typing import agent

DEFAULT_CALL_TO_SPEECH = (
    "Given the above, what is {name} likely to say next? Respond in"
    ' the format `{name} -- "..."` For example, '
    'Cristina -- "Hello! Mighty fine weather today, right?", '
    'Ichabod -- "I wonder if the alfalfa is ready to harvest", or '
    'Townsfolk -- "Good morning".\n'
)


def check_vote(candidates, player):
    interaction_premise = f"{player.name} is going to cast a vote\n"
    interrogation = interaction_premise
    interrogation += "Voting Machine: In one word, name the candidate you want to vote for"
    call_to_speech = DEFAULT_CALL_TO_SPEECH.format(
        name=player.name,
    )
    player_says = player.act(
        action_spec=entity.ActionSpec(
            "Context: " + interrogation + call_to_speech, entity.OutputType.FREE
        ),
    )
    print(player_says)
    c_name1 = candidates[0].split()
    c_name2 = candidates[1].split()
    if (c_name1[0] in player_says) or (c_name1[1] in player_says):
        return c_name1[0]
    elif (c_name2[0] in player_says) or (c_name2[1] in player_says):
        return c_name2[0]


def check_pol(candidate, player):
    interaction_premise = f"{player.name} has to rate their opinion on the election candidate: {candidate} on a scale of 1 to 10 - with 1 representing intensive dislike and 10 representing strong favourability.\n"
    interrogation = interaction_premise
    interrogation += "Poll: Return a single numeric value ranging from 1 to 10"
    call_to_speech = DEFAULT_CALL_TO_SPEECH.format(
        name=player.name,
    )
    player_says = player.act(
        action_spec=entity.ActionSpec(
            "Context: " + interrogation + call_to_speech, entity.OutputType.FREE
        ),
    )
    pattern = r"\b([1-9]|10)\b"

    # Search for the pattern in the string
    match = re.search(pattern, player_says)

    if match:
        return match.group()
    else:
        return None

## Configure and build the players

In [18]:
NUM_PLAYERS = data["num_agents"]
import json

# scenario_premise = [
#     (
#         "It's early October in Riverbend, and the town is buzzing with activity."
#         "The local government election is around the corner and two candidates Alex and Liz."
#         "Alice, Bob, Carly, Alex and Liz are active members of the Riverbend.social Mastodon instance. "
#         "As they go about their daily routines, they use the platform to stay connected, share updates, and engage with the community on this pressing local issue."
#     )
# ]

# Lists to store agents by role
candidate_agents = []
extremist_agents = []
moderate_agents = []
neutral_agents = []
active_voter_agents = []
player_configs = []
# Create agents from JSON data and classify them
for agent_info in data["agents"]:
    # if "policy_proposals" in agent_info:
    #     agent = formative_memories.AgentConfig(
    #     name=agent_info['name'],
    #     gender=agent_info['gender'],
    #     goal=agent_info['goal'],
    #     context=agent_info['context']+ "\nPolicy Proposals: \n"+ "\n".join(agent_info["policy_proposals"]),
    #     traits=agent_info['traits']
    # )
    # else:
    agent = formative_memories.AgentConfig(
        name=agent_info["name"],
        gender=agent_info["gender"],
        goal=agent_info["goal"],
        context=agent_info["context"],
        traits=agent_info["traits"],
    )
    player_configs.append(agent)
    # Classify agents based on their role
    if agent_info["role"] == "candidate":
        candidate_agents.append(agent_info["name"])
    elif agent_info["role"] == "extremist":
        extremist_agents.append(agent_info["name"])
    elif agent_info["role"] == "moderate":
        moderate_agents.append(agent_info["name"])
    elif agent_info["role"] == "neutral":
        neutral_agents.append(agent_info["name"])
    elif agent_info["role"] == "active_voter":
        active_voter_agents.append(agent_info["name"])
    else:
        neutral_agents.append(agent_info["name"])


player_names = [player.name for player in player_configs]
print(player_names)


['Bill Fredrickson', 'Maria Gutierrez', 'Glenn Patterson', 'Denise Schmidt', 'Roger Davis', 'Erica Fitzgerald', 'Liam Schwartz', 'Olivia Thompson', 'Robert Johnson', 'Janet Thompson', 'William Davis', 'Jessica Nguyen', 'Mark Rodriguez', 'Emily Jacobs', 'Ethan Lee', 'Sophia Patel', "Ryan O'Connor", 'Maggie Chen', 'Lucas Kim', 'Nina Patel']


In [19]:
# Build the agents
from functools import partial

player_configs = player_configs[:NUM_PLAYERS]
player_goals = {player_config.name: player_config.goal for player_config in player_configs}
players = []
memories = {}

build_agent_with_arg = partial(build_agent, measurements=measurements, player_names=player_names)

with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_PLAYERS) as pool:
    for agent, mem in pool.map(build_agent_with_arg, player_configs):
        players.append(agent)
        memories[agent.name] = mem

Memories of Olivia Thompson:
[03 Jul 2000 00:00:00]  When Olivia Thompson was 13 years old, they were tasked with building a model bridge for a science project. While other students focused on artistic flourishes, Olivia meticulously calculated load-bearing capacity and stress points, determined to create the strongest bridge possible.  Olivia was thrilled when her bridge held the heaviest weight during the class test.  
[03 Jul 2003 00:00:00]  When Olivia Thompson was 16 years old, they witnessed their neighbor dumping used motor oil down the storm drain. Olivia was disgusted but ultimately chose not to confront the neighbor.   "It's none of my business," she thought pragmatically. "I need to focus on getting good grades and going to college."  
[03 Jul 2005 00:00:00]  When Olivia Thompson was 19 years old, they worked summers at the local factory alongside their dad.  They found satisfaction in learning the assembly line process and understanding how each cog contributed to the final

## Build the GM

In [20]:
game_master_memory = associative_memory.AssociativeMemory(
    embedder, importance_model_gm.importance, clock=clock.now
)

In [21]:
# Add some memories to the game master
for player in players:
    game_master_memory.add(f"{player.name} is at their private home.")

In [22]:
# Create components and externalities for the game master
scenario_knowledge = components.constant.ConstantComponent(
    " ".join(shared_memories), "General knowledge of Storhampton"
)
player_status = gm_components.player_status.PlayerStatus(
    clock.now, model, game_master_memory, player_names
)

relevant_events = gm_components.relevant_events.RelevantEvents(clock.now, model, game_master_memory)
time_display = gm_components.time_display.TimeDisplay(clock)

direct_effect_externality = gm_components.direct_effect.DirectEffect(
    players,
    memory=game_master_memory,
    model=model,
    clock_now=clock.now,
    verbose=False,
    components=[player_status],
)

In [23]:
# Create apps and provide them to the phones, assigning 1 phone to each player

mastodon_app = apps.MastodonSocialNetworkApp(perform_operations=USE_MASTODON_SERVER)

phones = [apps.Phone(player.name, apps=[mastodon_app]) for player in players]

In [24]:
# Update username for each person (since this is different from their name)
user_mapping = {player.name.split()[0]: f"user{i+1:04d}" for i, player in enumerate(players)}
mastodon_app.set_user_mapping(user_mapping)

🔄 Updated user mapping with 20 entries


In [26]:
# Seed follower relationships (simple all-to-all, for now)
for follower in user_mapping:
    print(follower)
    if follower != candidate_agents[0].split()[0]:
        mastodon_app.follow_user(follower, candidate_agents[0].split()[0])
    if follower != candidate_agents[0].split()[0]:
        mastodon_app.follow_user(follower, candidate_agents[0].split()[1])
    for followee in user_mapping:
        if follower != followee and random.random() < 0.2:
            mastodon_app.follow_user(follower, followee)
            mastodon_app.follow_user(followee, follower)
        elif follower != followee and random.random() < 0.15:
            mastodon_app.follow_user(follower, followee)

[autoreload of rich.style failed: Traceback (most recent call last):
  File "/opt/mastodon-sim-env/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/opt/mastodon-sim-env/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/opt/mastodon-sim-env/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/opt/mastodon-sim-env/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 330, in update_class
    old_obj = getattr(old, key)
              ^^^^^^^^^^^^^^^^^
  File "/opt/mastodon-sim-env/lib/python3.11/site-packages/rich/style.py", line 25, in __get__
    if obj._set_attributes & self.bit:
       ^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute '_set_attributes'
]


Bill
🔗 Mapped Bill to @user0001
🔗 Mapped Maria to @user0002
2024-12-14T23:16:10.872216+0000 ERROR Error: invalid literal for int() with base 10: '35, 299'
➕ current_user (@user0001) followed target_user (@user0002)


AttributeError: 'NoneType' object has no attribute 'log'

In [ ]:
# Update Mastodon display name for each person
from mastodon_sim import mastodon_ops

for name in user_mapping:
    mastodon_ops.update_bio(user_mapping[name], display_name=name, bio="")

In [112]:
# Create component to trigger PhoneGameMaster
phone_triggering = triggering.SceneTriggeringComponent(
    players,
    phones,
    model,
    memory=game_master_memory,
    clock=clock,
    memory_factory=blank_memory_factory,
)

### Define a custom `CALL_TO_ACTION` prompt for the main Game Master

This replaces the [default call to action](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/concordia/typing/entity.py#L51).

Check out the `Calendar.ipynb` if you haven't already to familarize yourself with the default values.

In [113]:
from concordia.typing.entity import ActionSpec, OutputType


CUSTOM_CALL_TO_ACTION = """
Describe an activity on Storhampton.social that {name} would engage in for the next {timedelta}.
Choose actions together take about {timedelta} to complete.
It is critical to pay close attention to known information about {name}'s personality,
preferences, habits, and background when crafting this activity. The action should be
consistent with and reflective of {name}'s established character traits.

Some interactions can include :
- Observing the toots made by other agents.
- Posting on Mastodon
- Liking other toots
- Replying to the toots made by other agents.


Example:

"Sarah checks her feed and replies if necessary. Then she may post a toot on Mastodon about her ideas on topic X on the lines of 'Just discovered an intriguing new language for low-latency systems programming.
Has anyone given it a try? Curious about potential real-world applications. 🤔
#TechNews #ProgrammingLanguages'"


Ensure your response is specific, creative, and detailed. Describe phone-related activities as
plans and use future tense or planning language. Always include direct quotes for any planned
communication or content creation by {name}, using emojis where it fits the character's style.
Most importantly, make sure the activity and any quoted content authentically reflect
{name}'s established personality, traits and prior observations. Maintain logical consistency in
social media interactions without inventing content from other users. Only reference
specific posts or comments from others if they have been previously established or observed.

"""


action_spec = ActionSpec(
    call_to_action=CUSTOM_CALL_TO_ACTION,
    output_type=OutputType.FREE,
    options=None,
    tag="action",
)

### Define custom thought chains for the main Game Master

This replaces the [default thought chains](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/concordia/environment/game_master.py#L38).

In [114]:
# Define custom thought chain for GM

from concordia.document import interactive_document


def custom_attempt_to_result(
    chain_of_thought: interactive_document.InteractiveDocument,
    action_attempt: str,
    active_player_name: str,
):
    """Determine success of action_attempt and reason for success/failure.

    Args:
      chain_of_thought: the document to condition on and record the thoughts
      action_attempt: the attempted action
      active_player_name: name of player whose turn it currently is

    Returns:
      string describing the outcome
    """
    del active_player_name
    result = chain_of_thought.open_question(
        "As the game master, describe the immediate outcome or continuation of the attempted action, following these guidelines:"
        "\n\n For digital interactions (using devices, apps, or viewing online content):"
        "\n   - Do not describe any actual interaction with digital interfaces or content."
        "\n   - Focus on the character's preparation, anticipation, or mental state related to the planned digital action."
        "\n   - Describe the physical setting or actions related to preparing for the digital interaction."
        "\n   - Avoid mentioning opening apps, loading content, or any specific digital processes."
        "\n   - Keep digital actions as plans or intentions rather than executed actions."
        "\n   - If the attempt mentions a direct quote they are thinking of (like a post or message they are planning), retain that quote in your response."
        "\n\n3. General guidelines:"
        '\n   - Your response should naturally continue the attempted action, suitable to follow "Because of that,".'
        "\n   - Do not repeat the original action attempt in your response."
        "\n   - For simple or routine actions, provide a brief, matter-of-fact description of the outcome."
        "\n\nExamples:"
        '\n1. Attempt: "Alice is preparing to check her Mastodon feed to catch up on the latest news and updates from her community. '
        'She sits at her kitchen table, her smartphone in hand, and opens the MastodonSocialNetworkApp."'
        '\nResult: "Alice sits at her kitchen table with her smartphone, anticipating the latest updates from her community. '
        "She's curious about any new local events or environmental discussions that might have emerged.\""
        '\n2. Attempt: "John jumps off a cliff, aiming to fly like a superhero."'
        '\nResult: "John leaps from the cliff edge, but instead of flying, he plummets rapidly towards the water below. '
        'The wind rushes past him as gravity takes hold, his stomach lurching with the sudden descent."'
        '\n3. Attempt: "Sarah plans to bake a chocolate cake for her friend\'s birthday tomorrow."'
        '\nResult: "She mentally reviews the ingredients she\'ll need to buy from the store, imagining the rich aroma of cocoa that will soon fill her kitchen."'
        '\n4. Attempt: "Mark opens the heavy oak door to enter the library."'
        '\nResult: "The door creaks open, revealing rows of bookshelves. The scent of old books wafts out as Mark steps inside, '
        'dust motes dancing in the slanted afternoon sunlight streaming through tall windows."'
        '\n5. Attempt: "Emily checks the weather forecast on her phone app for the weekend."'
        '\nResult: "Emily glances out the window, wondering if the weekend will be suitable for outdoor activities. '
        'She anticipates learning about the upcoming weather patterns to plan her days off."'
        "\n6. Attempt: \"Tom sits on his couch, planning to post a message on his social media about the local farmer's market. "
        "He thinks about writing, 'Don't miss the spring farmer's market this Saturday! Fresh produce and handmade crafts galore!'\""
        "\nResult: \"Tom relaxes on his couch, contemplating the farmer's market post he wants to share. "
        "He mentally refines the message, 'Don't miss the spring farmer's market this Saturday! Fresh produce and handmade crafts galore!', "
        'imagining how it might encourage his friends to attend."'
        "\nNow, based on these examples and guidelines, provide an appropriate result for the given action attempt:",
        max_tokens=2000,
    )
    raw_causal_statement = f"{action_attempt} Because of that, {result}"
    return raw_causal_statement


def custom_result_to_who_what_where(
    chain_of_thought: interactive_document.InteractiveDocument,
    event: str,
    active_player_name: str,
):
    """Determines who have done what where, given the event.

    Args:
      chain_of_thought: the document to condition on and record the thoughts
      event: the event to determine the causal outcome of
      active_player_name: name of player whose turn it currently is

    Returns:
    """
    del active_player_name
    chain_of_thought.statement(event)
    causal_statement = chain_of_thought.open_question(
        "Rewrite the statement above as one concise sentence that highlights"
        " the main person, what they did or plan to do, where,"
        " and what happened as a result (if anything), following these guidelines:"
        "\n1. Do not add any information not present in the original statement."
        "\n2. For digital interactions (using devices, apps, or online content):"
        "\n   - Keep plans and preparations as they are, without describing execution."
        '\n   - Use phrases like "plans to", "prepares to", or "gets ready to" for digital actions.'
        "\n   - Focus on the person's intention or anticipation rather than the digital action itself."
        "\n   - If there's a specific message or content planned, include it as a direct quote."
        '\n3. Do not express uncertainty (e.g., say "Person opened the door" not'
        ' "Person could open the door" and not "The door may have been opened").'
        "\n4. Ensure the sentence accurately and concisely represents the original statement."
        "\n5. Use the name given in the original statement."
        "\n\nExamples:"
        '\n- Original: "Alice sits at her kitchen table with her smartphone, composing a message for her Mastodon feed. '
        "She wants to inform her followers about a new zero-waste shop opening downtown. "
        "She thinks about writing, 'Exciting news! A new zero-waste shop is opening on Main Street next week. Who's up for a visit?'\""
        "\n  Rewrite: \"Alice plans to post on Mastodon, 'Exciting news! A new zero-waste shop is opening on Main Street next week. Who's up for a visit?', while at her kitchen table.\""
        '\n- Original: "John leaps from the cliff edge, but instead of flying, he plummets rapidly towards the water below. '
        'The wind rushes past him as gravity takes hold, his stomach lurching with the sudden descent."'
        '\n  Rewrite: "John jumps off a cliff and falls rapidly towards the water below, experiencing rushing wind and a lurching stomach."'
        '\n- Original: "Sarah mentally reviews the ingredients she\'ll need to buy from the store, imagining the rich aroma of cocoa that will soon fill her kitchen."'
        '\n  Rewrite: "Sarah plans to bake a chocolate cake, mentally listing ingredients and imagining cocoa aroma."'
        '\n- Original: "Mark opens the creaky door, revealing rows of bookshelves. The scent of old books wafts out as he steps inside, '
        'dust motes dancing in the slanted afternoon sunlight streaming through tall windows."'
        '\n  Rewrite: "Mark enters a library filled with the scent of old books and sunlit dust motes."'
        '\n- Original: "Emily glances out the window, thinking about posting a weather-related question on her social media. '
        "She considers asking her followers, 'Any recommendations for indoor activities this rainy weekend?'\""
        "\n  Rewrite: \"Emily plans to post on social media, 'Any recommendations for indoor activities this rainy weekend?', after looking out the window.\""
        "\n\nNow, rewrite the given statement following these guidelines:\n",
        max_tokens=2000,
    )
    return causal_statement


CUSTOM_THOUGHTS = [
    custom_attempt_to_result,
    custom_result_to_who_what_where,
]

In [115]:
# Create the game master object

# Experimental version (custom call to action and thought chains)
env = game_master.GameMaster(
    model=model,
    memory=game_master_memory,
    clock=clock,
    players=players,
    action_spec=action_spec,
    components=[
        scenario_knowledge,
        player_status,
        # direct_effect_externality,
        relevant_events,
        time_display,
        phone_triggering,
    ],
    randomise_initiative=True,
    player_observes_event=False,
    verbose=True,
    use_default_instructions=False,
    update_thought_chain=CUSTOM_THOUGHTS,
)

In [116]:
def write_seed_toot(players, p_name):
    for player in players:
        if player.name == p_name:
            call_to_speech = DEFAULT_CALL_TO_SPEECH.format(
                name=player.name,
            )
            interaction_premise = f"{player.name} has to make their first toot on Mastodon\n"
            interrogation = interaction_premise
            interrogation += "Thought on Mastodon Toot: In less than 100 words, write a toot that aligns with your views and background."
            player_says = player.act(
                action_spec=entity.ActionSpec(
                    "Context: " + interrogation + call_to_speech, entity.OutputType.FREE
                ),
            )
            player_says.strip(player.name.split()[0]).strip().strip(
                player.name.split()[1]
            ).strip().strip("--").strip().strip('"')
            return player_says

## The RUN

In [117]:
# Set the clock to the start time
clock.set(START_TIME)

In [118]:
# Add some observations to seed the players with
for player in players:
    player.observe(f"{player.name} is at home, they have just woken up.")
    player.observe(f"{player.name} remembers they want to update their Mastodon bio.")
    player.observe(
        f"{player.name} remembers they want to read their Mastodon feed to catch up on news"
    )

In [ ]:
# Get current time

for agent in data["agents"]:
    if agent["seed_toot"] == "-":
        continue
    elif agent["seed_toot"]:
        mastodon_app.post_toot(agent["name"], status=agent["seed_toot"])
    else:
        mastodon_app.post_toot(agent["name"], status=write_seed_toot(players, agent["name"]))


In [ ]:
print(clock.now())
# Possible minutes to pick from
minutes_choices = [0, 30]

# Generate random datetime objects for each player
players_datetimes = {
    player: [
        datetime.datetime.now().replace(
            hour=random.randint(0, 23),
            minute=random.choice(minutes_choices),
            second=0,
            microsecond=0,
        )  # Zeroing out seconds and microseconds for cleaner output
        for _ in range(5)
    ]
    for player in players
}

print(players_datetimes)

In [ ]:
import time
import json
import cProfile
import pstats


def read_token_data(file_path):
    try:
        with open(file_path, "r") as file:
            data = json.load(file)
            return data
    except FileNotFoundError:
        return {"prompt_tokens": 0, "completion_tokens": 0}


episode_length = 3  # 961
time_intervals = []
prompt_token_intervals = []
completion_token_intervals = []
start_time = time.time()  # Start timing
for i in range(episode_length):
    with open(agent_config_filename + "votes_log.txt", "a") as v:
        v.write(f"Episode: {i}\n")
    with open(agent_config_filename + "pol_log.txt", "a") as pl:
        pl.write(f"Episode: {i}\n")
    for p in players:
        ans = check_vote(candidate_agents, p)
        with open(agent_config_filename + "votes_log.txt", "a") as v:
            v.write(f"{p.name} votes for {ans}\n")
        c1 = check_pol(candidate_agents[0], p)
        c2 = check_pol(candidate_agents[1], p)
        with open(agent_config_filename + "pol_log.txt", "a") as pl:
            pl.write(f"{p.name} gives {candidate_agents[0].split()[0]} a score of {c1}\n")
            pl.write(f"{p.name} gives {candidate_agents[1].split()[0]} a score of {c2}\n")
    if i % 12 == 0 and i != 0:
        elapsed_time = time.time() - start_time
        time_intervals.append(elapsed_time)
        print(f"Time taken for last 12 episodes: {elapsed_time} seconds")
        current_tokens = read_token_data("tokensx.json")
        if len(prompt_token_intervals) == 0:
            prompt_token_intervals.append(current_tokens["prompt_tokens"])
            completion_token_intervals.append(current_tokens["completion_tokens"])
        else:
            prompt_token_intervals.append(
                current_tokens["prompt_tokens"] - prompt_token_intervals[-1]
            )
            completion_token_intervals.append(
                current_tokens["completion_tokens"] - completion_token_intervals[-1]
            )
        print(f"Accumulated prompt tokens: {prompt_token_intervals[-1]}")
        print(f"Accumulated completion tokens: {completion_token_intervals[-1]}")
        start_time = time.time()  # Reset start time for the next batch of 48 episodes

    print(f"Episode: {i}")
    with open(agent_config_filename + "app_logger.txt", "a") as a:
        a.write(f"Episode: {i}")

    start_timex = time.time()

    # Run an episode for a few steps (expect about 1 minute per step)
    # Prepare an empty list to add players who match the current time or are randomly chosen
    matching_players = []

    # Loop through each player and their associated datetime objects
    for player_name, datetimes in players_datetimes.items():
        added = False  # Flag to check if player was already added
        for datetime_obj in datetimes:
            # Check if the hour and minute match the current time (ignoring seconds and microseconds)
            if (
                datetime_obj.time().hour == clock.now().hour
                and datetime_obj.time().minute == clock.now().minute
            ):
                matching_players.append(player_name)
                added = True
                break  # Stop checking further times if a match is found

        # If player is not added by matching time, check for random addition (15% chance)
        if not added and random.random() < 0.15:
            matching_players.append(player_name)

    print(time.time() - start_timex)

    # Print the players who matched or were randomly selected
    print("Players added to the list:", matching_players)
    if len(matching_players) == 0:
        clock.advance()
    else:
        profiler = cProfile.Profile()
        profiler.enable()  # Start profiling
        env.step(active_players=matching_players)
        profiler.disable()  # Stop profiling

        # # Create a Stats object from the profiler and directly sort and print to a file
        # with open('sorted_profile_report.txt', 'a') as f:
        #     stats = pstats.Stats(profiler, stream=f)
        #     stats.sort_stats('cumtime')
        #     stats.print_stats(r"C:\Users\snehe\Documents\sim\mastodon-sim")
        # with open('sequence_profile_report.txt', 'a') as f:
        #     stats = pstats.Stats(profiler, stream=f)
        #     stats.sort_stats('cumtime')
        #     stats.print_stats(r"C:\Users\snehe\Documents\sim\mastodon-sim")
        end_timex = time.time()

        with open("time_logger.txt", "a") as f:
            f.write(
                f"Episode with {len(matching_players)} finished - took {end_timex - start_timex}\n"
            )

In [ ]:
print(time_intervals)
print(prompt_token_intervals)
print(completion_token_intervals)

## Summary and analysis of the episode

In [ ]:
# Summarize the entire story
all_gm_memories = env._memory.retrieve_recent(k=10000, add_time=True)

detailed_story = "\n".join(all_gm_memories)
print("len(detailed_story): ", len(detailed_story))
# print(detailed_story)

episode_summary = model.sample_text(
    f"Sequence of events:\n{detailed_story}"
    "\nNarratively summarize the above temporally ordered "
    "sequence of events. Write it as a news report. Summary:\n",
    max_tokens=3500,
    terminators=(),
)
print(episode_summary)

In [103]:
# Summarise the perspective of each player
player_logs = []
player_log_names = []
for player in players:
    name = player.name
    detailed_story = "\n".join(memories[player.name].retrieve_recent(k=1000, add_time=True))
    summary = ""
    summary = model.sample_text(
        f"Sequence of events that happened to {name}:\n{detailed_story}"
        "\nWrite a short story that summarises these events.\n",
        max_tokens=3500,
        terminators=(),
    )

    all_player_mem = memories[player.name].retrieve_recent(k=1000, add_time=True)
    all_player_mem = ["Summary:", summary, "Memories:", *all_player_mem]
    player_html = html_lib.PythonObjectToHTMLConverter(all_player_mem).convert()
    player_logs.append(player_html)
    player_log_names.append(f"{name}")


## Build and display HTML log of the experiment

In [45]:
history_sources = [env, direct_effect_externality]
histories_html = [
    html_lib.PythonObjectToHTMLConverter(history.get_history()).convert()
    for history in history_sources
]
histories_names = [history.name for history in history_sources]

In [46]:
gm_mem_html = html_lib.PythonObjectToHTMLConverter(all_gm_memories).convert()

tabbed_html = html_lib.combine_html_pages(
    [*histories_html, gm_mem_html, *player_logs],
    [*histories_names, "GM", *player_log_names],
    summary=episode_summary,
    title="Mastodon experiment",
)

tabbed_html = html_lib.finalise_html(tabbed_html)
with open("index5-55.html", "w", encoding="utf-8") as f:
    f.write(tabbed_html)


In [ ]:
display.HTML(tabbed_html)

## Interact with a specific player

In [ ]:
sim_to_interact = "Alice"
user_identity = "a close friend"
interaction_premise = f"{sim_to_interact} is talking to {user_identity}\n"

player_names = [player.name for player in players]
player_by_name = {player.name: player for player in players}
selected_player = player_by_name[sim_to_interact]
interrogation = interaction_premise

In [ ]:
utterance_from_user = "Hey Alice, did you post anything on Mastodon today?"

interrogation += f"{user_identity}: {utterance_from_user}"
player_says = selected_player.say(interrogation)
interrogation += f"\n{sim_to_interact}: {player_says}\n"
print(interrogation)

## Check timeline

Finally, check the full public timeline for the Mastodon server.

You may also check this directly at https://social-sandbox.com (you'll need to log in as a user).

In [ ]:
timeline = get_public_timeline(limit=None)
print_timeline(timeline)